In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from tqdm import tqdm

### Hyperparameters and Device Configuration

In [5]:
DATA_PATH = "D:/PBL-Crop-PlantVillage Dataset/plantvillage dataset/color"
BATCH_SIZE = 16
IMG_SIZE = 224
EPOCHS = 10
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VERBOSE = 1  # Set to 0 to disable progress bars

print(f"Using device: {DEVICE}")

Using device: cuda


### Check if the dataset exists

In [7]:
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset folder missing: {DATA_PATH}")
print(f"Dataset found at: {DATA_PATH}")

Dataset found at: D:/PBL-Crop-PlantVillage Dataset/plantvillage dataset/color


### Define the data transforms with proper normalization for RGB images

In [9]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
print("Data transforms defined.")

Data transforms defined.


### Load the full dataset using ImageFolder

In [11]:
dataset = ImageFolder(root=DATA_PATH, transform=transform)
print(f"Dataset loaded with {len(dataset)} samples across {len(dataset.classes)} classes.")

Dataset loaded with 54305 samples across 38 classes.


### Split the dataset into training, validation, and test sets (70% / 15% / 15%)

In [13]:
total_len = len(dataset)
train_len = int(0.7 * total_len)
val_len = int(0.15 * total_len)
test_len = total_len - train_len - val_len

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_len, val_len, test_len])
print(f"Dataset split into {len(train_dataset)} training, {len(val_dataset)} validation, and {len(test_dataset)} test samples.")

Dataset split into 38013 training, 8145 validation, and 8147 test samples.


### Create data loaders

In [15]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print("Data loaders created.")

Data loaders created.


### Load a pretrained ResNet50 model and update the final fully connected layer

In [17]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))
model = model.to(DEVICE)
print("Model initialized and modified for the dataset.")
print(model)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model initialized and modified for the dataset.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0):

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print("Loss function and optimizer defined.")

Loss function and optimizer defined.


### Evaluation function that computes both loss and accuracy

In [20]:
def evaluate_model(model, data_loader, criterion, verbose=1):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    # Use tqdm for progress feedback during evaluation
    eval_progress = tqdm(data_loader, desc="Evaluating", leave=False, disable=verbose==0)
    with torch.no_grad():
        for images, labels in eval_progress:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = running_loss / len(data_loader)
    acc = correct / total
    return avg_loss, acc

### Training loop that displays metrics while training

In [22]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, verbose=1):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        batch_count = 0

        train_progress = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False, disable=verbose==0)
        for images, labels in train_progress:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            batch_count += 1
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Update progress bar with training metrics
            if verbose:
                train_progress.set_postfix(loss=running_loss / batch_count, acc=correct / total)

        # Evaluate on the validation set at the end of each epoch
        val_loss, val_acc = evaluate_model(model, val_loader, criterion, verbose)
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

### Train the model

In [24]:
print("Starting training...")
train_model(model, train_loader, val_loader, criterion, optimizer, EPOCHS, verbose=VERBOSE)
print("Training completed.")

Starting training...


Epoch 1/10 - Train Loss: 0.6572, Train Acc: 0.8021 | Val Loss: 0.2577, Val Acc: 0.9202


Epoch 2/10 - Train Loss: 0.2241, Train Acc: 0.9278 | Val Loss: 0.1711, Val Acc: 0.9432


Epoch 3/10 - Train Loss: 0.1612, Train Acc: 0.9468 | Val Loss: 0.2283, Val Acc: 0.9304


Epoch 4/10 - Train Loss: 0.1224, Train Acc: 0.9598 | Val Loss: 0.0874, Val Acc: 0.9694


...

Epoch 6/10 - Train Loss: 0.0874, Train Acc: 0.9711 | Val Loss: 0.0476, Val Acc: 0.9847


Epoch 7/10 - Train Loss: 0.0824, Train Acc: 0.9726 | Val Loss: 0.0696, Val Acc: 0.9783


Epoch 8/10 - Train Loss: 0.0674, Train Acc: 0.9777 | Val Loss: 0.0938, Val Acc: 0.9699


Epoch 9/10 - Train Loss: 0.0609, Train Acc: 0.9791 | Val Loss: 0.0528, Val Acc: 0.9849


Epoch 10/10 - Train Loss: 0.0578, Train Acc: 0.9807 | Val Loss: 0.0443, Val Acc: 0.9850
Training completed.


### Final evaluation on the test set

In [26]:
print("Evaluating on test set...")
test_loss, test_acc = evaluate_model(model, test_loader, criterion, verbose=VERBOSE)
print(f"Final Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Evaluating on test set...


Final Test Loss: 0.0381, Test Accuracy: 0.9880


### Save the trained model

In [46]:
torch.save(model, 'trained_plant_disease_model_complete.pth')
print(f"Model saved to: {model_path}")

Model saved to: trained_plant_disease_model.pth


In [48]:
dataset.classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_